# Trabalho Final

**Aluno**:

**Matrícula**:



## 05 - Tratamento da base *experiencia_trabalho.csv*

Descreva a base com suas palavras.

In [167]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

experiencia = pd.read_csv("bases/experiencia_trabalho.csv")

# Análise inicial
print("Nº de registros:", len(experiencia))
print("Nº de valores únicos de perfil_id:", experiencia.perfil_id.nunique())
print("Nº de valores únicos de company:", experiencia.company.nunique())
print("Nº de valores únicos de location:", experiencia.location.nunique())
print("Nº de valores únicos de position:", experiencia.position.nunique())
print("Nº de valores vazios de company:", experiencia.company.isnull().sum())
print("Nº de valores vazios de date:", experiencia.date.isnull().sum())
print("Nº de valores vazios de location:", experiencia.location.isnull().sum())
print("Nº de valores vazios de position:", experiencia.position.isnull().sum())

experiencia.describe(include="all")

Nº de registros: 497
Nº de valores únicos de perfil_id: 111
Nº de valores únicos de company: 2
Nº de valores únicos de location: 71
Nº de valores únicos de position: 386
Nº de valores vazios de company: 25
Nº de valores vazios de date: 26
Nº de valores vazios de location: 160
Nº de valores vazios de position: 25


,company,date,location,position,perfil_id
count,472,471,337,472,497.000000
unique,2,328,71,386,NaN
top,Any Company,mar de 2017 – o momento,"Belo Horizonte e Região, Brasil",Software Developer,NaN
freq,318,21,138,9,NaN
mean,NaN,NaN,NaN,NaN,54.187123
std,NaN,NaN,NaN,NaN,32.199717
min,NaN,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,NaN,27.000000
50%,NaN,NaN,NaN,NaN,54.000000
75%,NaN,NaN,NaN,NaN,81.000000


In [168]:
experiencia = experiencia[(experiencia.company.notnull()) & (experiencia.position.notnull()) & (experiencia.date.notnull())]
print("Nº de valores únicos de perfil_id:", experiencia.perfil_id.nunique())
print("Nº de registros restantes após remoção dos vazios:", len(experiencia))
print("Nº de valores vazios de company:", experiencia.company.isnull().sum())
print("Nº de valores vazios de date:", experiencia.date.isnull().sum())
print("Nº de valores vazios de location:", experiencia.location.isnull().sum())
print("Nº de valores vazios de position:", experiencia.position.isnull().sum())

Nº de valores únicos de perfil_id: 110
Nº de registros restantes após remoção dos vazios: 471
Nº de valores vazios de company: 0
Nº de valores vazios de date: 0
Nº de valores vazios de location: 134
Nº de valores vazios de position: 0


## 05.1 - Tratamento das data

A coluna **date** pode ser quebrada no ano de início (**inicio**) e no ano de fim (**fim**) da experiência. As experiências em aberto, possuem o texto "o momento" que será interpretado como o mês/ano da extração dos dados (09/2017). As experiências em aberto serão marcadas na coluna booleana **atual**, que indicando que o funcionário ainda trabalha na trojan.

In [170]:
experiencia.date = experiencia.date.str.replace('o momento','set de 2017')

ini_fim = experiencia.date.str.split('–')

experiencia['mesano_ini'] = ini_fim.str.get(0).str.strip()
experiencia['mesano_fim'] = ini_fim.str.get(1).str.strip()

#Cria a coluna atual para marcar apenas o emprego em aberto na empresa Trojan
experiencia['atual'] = (experiencia.mesano_fim.str.contains('set de 2017')) & (experiencia.company == 'Trojan Technologies')

#cria coluna para registrar o ano de inicio e fim do trabalho
experiencia['ano_ini'] = experiencia.mesano_ini.str.extract('(\d+)')
experiencia['ano_fim'] = experiencia.mesano_fim.str.extract('(\d+)')

A informação de duração da experiência (**duracao**) é derivada calculando a diferença do fim ao início em meses.

In [172]:
# calcula a duração estimada de cada experiência em meses

experiencia.mesano_ini = experiencia.mesano_ini.str.replace('jan','01') \
                        .str.replace('fev','02') \
                        .str.replace('mar','03') \
                        .str.replace('abr','04') \
                        .str.replace('mai','05') \
                        .str.replace('jun','06') \
                        .str.replace('jul','07') \
                        .str.replace('ago','08') \
                        .str.replace('set','09') \
                        .str.replace('out','10') \
                        .str.replace('nov','11') \
                        .str.replace('dez','12')

experiencia.mesano_fim = experiencia.mesano_fim.str.replace('jan','01') \
                        .str.replace('fev','02') \
                        .str.replace('mar','03') \
                        .str.replace('abr','04') \
                        .str.replace('mai','05') \
                        .str.replace('jun','06') \
                        .str.replace('jul','07') \
                        .str.replace('ago','08') \
                        .str.replace('set','09') \
                        .str.replace('out','10') \
                        .str.replace('nov','11') \
                        .str.replace('dez','12')

experiencia.loc[(experiencia.mesano_ini.str.len() == 4), 'mesano_ini'] = '01 de '+experiencia.mesano_ini
experiencia.loc[(experiencia.mesano_fim.str.len() == 4), 'mesano_fim'] = '01 de '+experiencia.mesano_fim 

experiencia['d_ini'] = pd.to_datetime(experiencia.mesano_ini, format='%m de %Y')
experiencia['d_fim'] = pd.to_datetime(experiencia.mesano_fim, format='%m de %Y')

experiencia['duracao_meses'] = experiencia.d_fim.dt.to_period('M') - experiencia.d_ini.dt.to_period('M')+1


Algumas informações consolidadas por funcionários podem ser extraídas dessa base. É possível extrair o tempo de carreira na empresa em meses (**tempo_empresa_meses**), o tempo de carreira no mercado em geral em meses (**tempo_mercado_meses**), e também a quantidade de empregos que o funcionário já teve (**qtd_empregos**), o que pode auxiliar em algo na análise. Portanto, obtenha essas informações.

In [198]:
# ----------- Cálculo do tempo_de_mercado_meses ------------ 
#     Os períodos de trabalho para um perfil não são subsequentes. Muitos períodos ocorrem simultaneamente.
#     Para cálculo do tempo de mercado foi utilizada uma estratégia de agrupar períodos sobrepostos em um mesmo período
#     contíno. Após esse agrupamento é realizado o cálculo de tempo de mercado para cada perfil

# Função utilizada para agrupar períodos contínuos e calcular o tempo de mercado em cada período
def calc_t_mercado_periodo(x):
     return pd.Series(dict(
                             perfil_id = x['perfil_id'].min(),
                             d_ini = x['d_ini'].min(), 
                             d_fim = x['d_fim'].max(),
                             tempo_meses = x['d_fim'].max().to_period('M') - x['d_ini'].min().to_period('M') +1
                          )
                     )
    
def calculo_meses(df):
#seleciona os códigos de perfil para iteração
    id_p = df['perfil_id'].unique()

#instancia dataframe que receberá os períodos de trabalho agrupados 
    g_periodos = pd.DataFrame()


#itera sobre cada perfil
    for i in id_p:
        p = df.loc[df.perfil_id == i]
        p = p.reset_index()

#itera sobre os periodos e verifica se o proximo periodo é sobreposto ao grupo de periodo anterior
#se não for registra, a data que sera usada para agrupar os periodos anteriores que sao sobrepostos 
        fim_grupo = p.loc[0, 'd_fim']
        for l in range(len(p)-1):
            if fim_grupo < p.loc[l+1, 'd_ini']:
                p.loc[l, 'grupo'] = p.loc[l, 'd_fim']

#se a data fim do periodo é maior que a data fim do proximo, mantem a data para comparacao       
            if fim_grupo < p.loc[l+1, 'd_fim']:
                fim_grupo = p.loc[l+1, 'd_fim']

#replica a mesma data final para todos os periodos que são sobrepostos
        l = p.tail(1).index
        p.loc[l,'grupo'] =  p.loc[l,'d_fim']
        p['grupo'] = p.grupo.fillna(method='backfill')

#agrupa os periodos e calcula o tempo de mercado para cada grupo de periodo
        p = p.groupby(['perfil_id','grupo'], as_index=False).apply(calc_t_mercado_periodo)
#adiciona periodos agrupados no dataframe
        g_periodos = pd.concat([g_periodos, p], ignore_index=True)
        
    return g_periodos

In [ ]:
#calcula tempo total de mercado por perfil
tempo_mercado_meses = experiencia[['perfil_id','d_ini','d_fim']].sort_values(by=['perfil_id','d_ini'])
tempo_mercado_meses = calculo_meses(tempo_mercado_meses)
tempo_mercado_meses = tempo_mercado_meses.groupby('perfil_id', as_index=False).agg({'tempo_meses':'sum'})

In [201]:
#calcula tempo_empresa_meses
tempo_empresa_meses = experiencia.loc[(experiencia.company == 'Trojan Technologies'), ['perfil_id','d_ini','d_fim']].sort_values(by=['perfil_id','d_ini'])
tempo_empresa_meses = calculo_meses(tempo_empresa_meses)
tempo_empresa_meses = tempo_empresa_meses.groupby('perfil_id', as_index=False).agg({'tempo_meses':'sum'})

In [234]:
#calcula quantidade de empregos
experiencia.groupby(['perfil_id']).position.count().to_frame()
#qtd_empregos = 

,position
perfil_id,
0,5
1,5
2,5
3,3
4,5
5,5
6,5
7,2
8,5


In [212]:

#tempo_mercado_meses = experiencia.groupby(['perfil_id']).duracao.sum().to_frame()
#qtd_empregos = experiencia.groupby(['perfil_id']).position.count().to_frame()

#qtd_empregos
#experiencia[(experiencia.atual) & (experiencia.company == 'Trojan Technologies') & ((experiencia.perfil_id == 6) | (experiencia.perfil_id == 11) | (experiencia.perfil_id == 12))]

,perfil_id,posicao
0,0,CEO and Founder
1,1,Product Manager
2,2,Full Stack Developer
3,3,CRM Analyst
4,4,Sales Operations Coordinator
5,5,Key Account Manager
6,6,"Software Engineer and Data Scientist, Data Eng..."
7,7,Aprediz administrativo
8,8,Founder and CFO
9,9,Sales Operations Manager


## 05.2 - Cargo atual na empresa


In [237]:
# Alguns perfis acumulam mais de uma posicao atualmente na empresa
# foi utilizada uma funcao para concatenar as posicoes acumulados em uma mesma linha do perfil


# Agrupa a posicao atual na emrepsa Trojan por perfil
cargo_atual = experiencia.loc[experiencia.atual] \
                        .groupby(['perfil_id']) \
                        .apply(lambda x: ', '.join(x['position'])).to_frame()
cargo_atual

,0
perfil_id,
0,CEO and Founder
1,Product Manager
2,Full Stack Developer
3,CRM Analyst
4,Sales Operations Coordinator
5,Key Account Manager
6,"Software Engineer and Data Scientist, Data Eng..."
7,Aprediz administrativo
8,Founder and CFO


## 05.3 - Setor do Funcionário na empresa

A partir da posicao dos funcionários, é possível descobrir (ou inferir) o setor que eles trabalham.

In [216]:
cargo_atual



,perfil_id,posicao
0,0,CEO and Founder
1,1,Product Manager
2,2,Full Stack Developer
3,3,CRM Analyst
4,4,Sales Operations Coordinator
5,5,Key Account Manager
6,6,"Software Engineer and Data Scientist, Data Eng..."
7,7,Aprediz administrativo
8,8,Founder and CFO
9,9,Sales Operations Manager


## 05.4 Salvando Base Tratada

A base tratada será gravada no arquivo **experiencia_trabalho-tratado.csv**.

As informações consolidadas por perfil serão gravadas na base **perfil-experiencia,csv** contendo as colunas: **perfil_id**, **posicao_atual**, **data_admissao**, **departamento**, **tempo_empresa_meses**, **tempo_mercado_meses**, **qtd_empregos**.

In [ ]:
# Grava um csv tratado

### Navegação:
* [Voltar para a análise](00-analise-dos-dados.ipynb)